In [ ]:
import os
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract, col
from delta import *

warehouse_diretory_path = '[YOUR_WAREHOUSE]'
ncbi_gene_accession_file = 'file:///[FILE_FULL_NAME:gene2accession.txt]'
biological_database_name = 'biological_database'
ncbi_gene_accession_table_name = 'bronze_ncbi_gene_accession'

conf = SparkConf()
conf.setAll(
    [
        ('spark.master', 'local[*]'), 
        ('spark.driver.host', 'localhost'),
        ('spark.app.name', 'NCBI Gene To Accession Importer'),
        ('spark.ui.showConsoleProgress', 'true'),
        ('spark.sql.execution.arrow.pyspark.enabled', 'true'),
        ('spark.driver.memory', '4g'),
        ('spark.executor.memory', '4g'),
        ('spark.sql.extensions','io.delta.sql.DeltaSparkSessionExtension'),
        ('spark.sql.catalog.spark_catalog', 'org.apache.spark.sql.delta.catalog.DeltaCatalog'),
        ('spark.sql.warehouse.dir', warehouse_diretory_path),
        ('spark.driver.extraJavaOptions', f'-Dderby.system.home={warehouse_diretory_path}')
    ])

spark = SparkSession.builder.config(conf=conf).enableHiveSupport().getOrCreate()

In [ ]:
ncbi_gene_accession_df = spark.read \
    .option('header', True) \
    .options(delimiter='\t') \
    .csv(ncbi_gene_accession_file)    

In [ ]:
spark.sql(f'CREATE DATABASE IF NOT EXISTS {biological_database_name};')

In [ ]:
spark.sql(f'USE {biological_database_name};')

In [ ]:
ncbi_gene_accession_df.write \
    .format('delta') \
    .mode('overwrite') \
    .option('overwriteSchema', 'true') \
    .option('partitionOverwriteMode', 'dynamic') \
    .partitionBy('`#tax_id`') \
    .saveAsTable(ncbi_gene_accession_table_name)

In [ ]:
spark.stop()